# Naive Bayes Implementation

Referencias:
- http://artificiencia.com/aprenda/algoritmo-naive-bayes/
- https://chrisalbon.com/machine_learning/naive_bayes/naive_bayes_classifier_from_scratch/
https://hackinganalytics.com/2016/10/13/desvendando-o-classificador-naive-bayes/


1. Dado um dataset, calcular probabilidade de cada classe
    - P(Jogar = Sim) = 9/14;
    - p(Jogar = Nao) = 5/14;
2. Calcular probabilidade de cada atributo
    - P(Tempo = Sol | Jogar = Sim) = 2/9;
    - Continua...
    
    
## Dúvidas
- Sobre o treinamento, o que é o treinamento do Navie Bayes.

In [4]:
import pandas as pd
import numpy as np

In [6]:
dataset = pd.read_csv('dataset/dataset.csv')
dataset

,Genero,Idade,Escolaridade,Profissao,Target
0,F,a - Ate 25 anos,Fundamental,b,0
1,M,a - Ate 25 anos,Medio,d,1
2,F,c - 36 a 45 anos,Fundamental,a,1
3,M,d - 46 a 55 anos,Fundamental,a,0
4,F,c - 36 a 45 anos,Fundamental,b,1
5,F,b - 26 a 35 anos,Fundamental,b,0
6,F,a - Ate 25 anos,Fundamental,b,0
7,F,e - Mais 56 anos,Superior,b,1
8,F,a - Ate 25 anos,Superior,b,1
9,M,b - 26 a 35 anos,Fundamental,a,1
